In [1]:
import pandas as pd
import re

In [2]:
''' Dados de entrada '''

data_path = '../data/'
projects_file = 'projetos_assembleia.csv'
dialogs_file = 'dialog_lines_database.csv'
#corruption_words = ['corrupto', 'corrupta', 'corrupção']
corruption_words = ['corrup']


In [3]:
projects = pd.read_csv(data_path+projects_file)
dialogs = pd.read_csv(data_path+dialogs_file)

print(projects['title'].unique())
#print(dialogs['Subject'].unique())
projects.head()

['Projeto de Resolução 123/VII' 'Projeto de Lei 637/VII'
 'Projeto de Lei 624/VII' 'Projeto de Lei 625/VII'
 'Apreciação Parlamentar 82/VII' 'Projeto de Resolução 122/VII'
 'Projeto de Lei 622/VII' 'Projeto de Lei 621/VII'
 'Projeto de Lei 617/VII' 'Projeto de Lei 620/VII'
 'Projeto de Lei 616/VII' 'Apreciação Parlamentar 96/VII'
 'Projeto de Lei 618/VII' 'Projeto de Lei 629/VII'
 'Projeto de Lei 619/VII' 'Apreciação Parlamentar 92/VII'
 'Projeto de Lei 627/VII' 'Projeto de Lei 664/VII'
 'Projeto de Lei 665/VII' 'Projeto de Lei 631/VII'
 'Projeto de Resolução 121/VII' 'Projeto de Lei 626/VII'
 'Projeto de Lei 652/VII' 'Projeto de Lei 615/VII'
 'Projeto de Lei 632/VII' 'Projeto de Resolução 126/VII'
 'Projeto de Lei 635/VII' 'Projeto de Lei 654/VII'
 'Apreciação Parlamentar 89/VII' 'Apreciação Parlamentar 87/VII'
 'Projeto de Lei 614/VII' 'Projeto de Lei 658/VII'
 'Projeto de Resolução 471/XIV' 'Projeto de Resolução 550/XIV']


,author,description,title,url
0,Luís Marques Mendes (PSD),Sobre a Agenda 2000,Projeto de Resolução 123/VII,https://www.parlamento.pt/ActividadeParlamenta...
1,"Luís Queiró (CDS-PP),Gonçalo Ribeiro da Costa ...",Conselhos municipais de acção social,Projeto de Lei 637/VII,https://www.parlamento.pt/ActividadeParlamenta...
2,"Pimenta Dias (PCP),Joaquim Matias (PCP),João A...",Isenta de emolumentos a apreciação das contas ...,Projeto de Lei 624/VII,https://www.parlamento.pt/ActividadeParlamenta...
3,"Luís Queiró (CDS-PP),Sílvio Rui Cervan (CDS-PP...","Pronuncia, acompanhamento e apreciação pela As...",Projeto de Lei 625/VII,https://www.parlamento.pt/ActividadeParlamenta...
4,Luís Queiró (CDS-PP),"Decreto-lei n° 15/99,de 15 de Janeiro, que""reg...",Apreciação Parlamentar 82/VII,https://www.parlamento.pt/ActividadeParlamenta...


In [41]:
subjects = []
last_subject = 'declarações políticas 0'
last_transcript = 'DAR-001'
j = 1
for i in range(len(dialogs['Text'])):
    name = dialogs['Person'][i]
    text = dialogs['Text'][i]
    transcript = dialogs['Transcript'][i]
    if(transcript != last_transcript):
        j = 0
        last_transcript = transcript
    lower_text = text.lower()
    if name == 'Presidente':
        if('declaraç' in lower_text):
            last_subject = 'declarações políticas {}'.format(j)
            j = j + 1            
        elif ('projeto de ' in lower_text and '/' in lower_text):
            positions = [m.start() for m in re.finditer('Projeto de', text)]
            last_subject = ''
            k = 0
            for position in positions:
                if (k == 0):
                    last_subject = text[position:text.find('/', position)+4].replace('n.º ', '')
                last_subject = last_subject + ', ' + text[position:text.find('/', position)+4].replace('n.º ', '')
                k = k + 1
    subjects.append(last_subject)
dialogs['Subject'] = subjects
dialogs

,Transcript,Date,Position,Person,Party,Text,Subject,key
0,DAR-001,17 DE SETEMBRO DE 2020,0,Presidente,NaN,Dirijo um abraço a todos neste regresso dos Pl...,declarações políticas 1,17 DE SETEMBRO DE 2020declarações políticas 1
1,DAR-001,17 DE SETEMBRO DE 2020,1,Secretária,Maria da Luz Rosinha,"Sr. Presidente, muito boa tarde a todas e a to...",declarações políticas 1,17 DE SETEMBRO DE 2020declarações políticas 1
2,DAR-001,17 DE SETEMBRO DE 2020,2,Presidente,NaN,"Como os Srs. Deputados estão a verificar, o an...",declarações políticas 1,17 DE SETEMBRO DE 2020declarações políticas 1
3,DAR-001,17 DE SETEMBRO DE 2020,3,Secretária,Maria da Luz Rosinha,"Sr. Presidente, deram também entrada os Projet...",declarações políticas 1,17 DE SETEMBRO DE 2020declarações políticas 1
4,DAR-001,17 DE SETEMBRO DE 2020,4,Presidente,NaN,"Muito obrigado, Sr.ª Secretária Maria da Luz R...",declarações políticas 2,17 DE SETEMBRO DE 2020declarações políticas 2
...,...,...,...,...,...,...,...,...
11300,DAR-047,26 DE FEVEREIRO DE 2021,184,Presidente,António Filipe,Muito obrigado.,"Projeto de Resolução 899/XIV, Projeto de Resol...",26 DE FEVEREIRO DE 2021Projeto de Resolução 89...
11301,DAR-047,26 DE FEVEREIRO DE 2021,185,Lara Martinho,PS,"Sr. Presidente, peço a palavra.","Projeto de Resolução 899/XIV, Projeto de Resol...",26 DE FEVEREIRO DE 2021Projeto de Resolução 89...
11302,DAR-047,26 DE FEVEREIRO DE 2021,186,Presidente,António Filipe,"Para que efeito, Sr.ª Deputada?","Projeto de Resolução 899/XIV, Projeto de Resol...",26 DE FEVEREIRO DE 2021Projeto de Resolução 89...
11303,DAR-047,26 DE FEVEREIRO DE 2021,187,Lara Martinho,PS,"Sr. Presidente, queria apenas informar a Mesa ...","Projeto de Resolução 899/XIV, Projeto de Resol...",26 DE FEVEREIRO DE 2021Projeto de Resolução 89...


In [42]:
def find_words(data_frame, fields, words):
    series = [data_frame[field] for field in fields]
    positions = pd.DataFrame()
    for serie in series:
        for word in words:
            positions[serie.name+'_'+word+"_pos"] = serie.str.lower().str.find(word)
    return positions

def get_citations(data_frame, fields, words):
    positions = find_words(data_frame, fields, words)
    out_list = [] # pd.DataFrame(columns = data_frame.columns)
    i = 0
    for ind in positions.index:
        has_found = False
        for field_word in positions:
            if (positions[field_word][ind] != -1):
                has_found = True
                break
        if(has_found):
            #print(data_frame.iloc[ind].values)
            out_list.append(data_frame.iloc[ind].values)
            i = i + 1
    return pd.DataFrame(out_list, columns = data_frame.columns)

dialogs_with_corruption_citations = get_citations(dialogs, ['Text'], corruption_words)
projects_with_corruption_citations = get_citations(projects, ['description'], corruption_words)

print('Diálogos que citam corrupção: ', len(dialogs_with_corruption_citations))
print('Projetos que citam corrupção: ', len(projects_with_corruption_citations))

print(dialogs_with_corruption_citations['Text'][2])
dialogs_with_corruption_citations

#for line in lines:
#    print(line)
#    print('------------')



Diálogos que citam corrupção:  89
Projetos que citam corrupção:  1
Esperei tantos meses, tantos meses pelo tal plano contra a corrupção que a Ministra da Justiça ia apresentar! E lá apresentou. Vejam que, ontem, conhecemos finalmente a realidade deste plano. Cito a Ministra da Justiça: «O grande ponto é a falta de dados e a falta de um sistema harmónico de dimensão preventiva e repressiva». Era isto que faltava, de facto. Olhando para o que precisamos, era aqui que estava a solução. Andámos enganados tanto tempo e estava nestas simples palavras a solução para a corrupção! Mas o que se podia esperar do partido que, um dia, foi liderado pelo Eng.º Guterres, ou que teve grandes nomes, como o de Armando Vara?! Não podíamos esperar muito mais. No meio disto e da inexistência de um plano contra a corrupção, temos um Presidente que diz que temos de apurar: temos de apurar Pedrógão, temos de apurar Tancos, temos de apurar o Novo Banco, temos de apurar os negócios familiares de membros do Gover

,Transcript,Date,Position,Person,Party,Text,Subject,key
0,DAR-001,17 DE SETEMBRO DE 2020,231,Filipa Roseta,PSD,O Prof. António Costa Silva disse hoje de manh...,declarações políticas 6,17 DE SETEMBRO DE 2020declarações políticas 6
1,DAR-001,17 DE SETEMBRO DE 2020,233,Filipa Roseta,PSD,… e que de cada vez que há um problema e há um...,declarações políticas 6,17 DE SETEMBRO DE 2020declarações políticas 6
2,DAR-001,17 DE SETEMBRO DE 2020,277,André Ventura,CH,"Esperei tantos meses, tantos meses pelo tal pl...",declarações políticas 8,17 DE SETEMBRO DE 2020declarações políticas 8
3,DAR-001,17 DE SETEMBRO DE 2020,309,Presidente,Fernando Negrão,"Damos, assim, por terminada a discussão do seg...",,17 DE SETEMBRO DE 2020
4,DAR-001,17 DE SETEMBRO DE 2020,310,André Silva,PAN,"Sr. Presidente, Sr.as e Srs. Deputados: Começo...",,17 DE SETEMBRO DE 2020
...,...,...,...,...,...,...,...,...
84,DAR-043,30 DE JANEIRO DE 2021,75,Ministro de Estado e dos Negócios Estrangeiros,NaN,A minha ignorância está em que eu também pensa...,declarações políticas 1,30 DE JANEIRO DE 2021declarações políticas 1
85,DAR-043,30 DE JANEIRO DE 2021,201,Isabel Pires,BE,"Sr. Presidente, Sr.as e Srs. Deputados: A Efac...",declarações políticas 1,30 DE JANEIRO DE 2021declarações políticas 1
86,DAR-045,12 DE FEVEREIRO DE 2021,126,André Silva,PAN,"Boa tarde, Sr. Presidente e Sr.as e Srs. Deput...","Projeto de Lei 615/XIV, Projeto de Lei 615/XIV","12 DE FEVEREIRO DE 2021Projeto de Lei 615/XIV,..."
87,DAR-045,12 DE FEVEREIRO DE 2021,132,Duarte Alves,PCP,"Sr. Presidente, Srs. Deputados, Srs. Membros d...","Projeto de Lei 615/XIV, Projeto de Lei 615/XIV","12 DE FEVEREIRO DE 2021Projeto de Lei 615/XIV,..."


In [43]:
def get_related(full_dataframe, citation_dataframe, key_fields):
    citation_dataframe['key'] = citation_dataframe[key_fields[0]]
    full_dataframe['key'] = full_dataframe[key_fields[0]]
    for i in range(1, len(key_fields)):
        citation_dataframe['key'] = citation_dataframe['key'] + citation_dataframe[key_fields[i]]
        full_dataframe['key'] = full_dataframe['key'] + full_dataframe[key_fields[i]]
    topics = citation_dataframe['key'].unique()
    #print(topics)
    to_out = pd.DataFrame(columns = full_dataframe.columns)
    for topic in topics:
        to_out = to_out.append(full_dataframe.loc[full_dataframe['key'] == topic])
    return to_out
    
dialogs_related_to_corruption = get_related(dialogs, dialogs_with_corruption_citations, ['Date', 'Subject'])
print(len(dialogs_related_to_corruption))
print(dialogs_related_to_corruption['key'].unique())
dialogs_related_to_corruption.head()
#print(related)

2886
['17 DE SETEMBRO DE 2020declarações políticas 6'
 '17 DE SETEMBRO DE 2020declarações políticas 8' '17 DE SETEMBRO DE 2020'
 '18 DE SETEMBRO DE 2020Projeto de Resolução 471/XIV, Projeto de Resolução 471/XIV, Projeto de Lei 499/XIV'
 '19 DE SETEMBRO DE 2020Projeto de Resolução 550/XIV, Projeto de Resolução 550/XIV, Projeto de Resolução 546/XIV, Projeto de Resolução 561/XIV, Projeto de Lei 474/XIV'
 '19 DE SETEMBRO DE 2020Projeto de Resolução 545/XIV, Projeto de Resolução 545/XIV, Projeto de Resolução 569/XIV, Projeto de Resolução 578/XIV, Projeto de Resolução 426/XIV'
 '19 DE SETEMBRO DE 2020Projeto de Resolução 265/XIV, Projeto de Resolução 265/XIV'
 '24 DE SETEMBRO DE 2020Projeto de Resolução 265/XIV, Projeto de Resolução 265/XIV'
 '26 DE SETEMBRO DE 2020Projeto de Resolução 553/XIV, Projeto de Resolução 553/XIV, Projeto de Deliberação 10/XIV'
 '1 DE OUTUBRO DE 2020declarações políticas 3'
 '1 DE OUTUBRO DE 2020declarações políticas 6'
 '3 DE OUTUBRO DE 2020Projeto de Resolução 63

,Transcript,Date,Position,Person,Party,Text,Subject,key
216,DAR-001,17 DE SETEMBRO DE 2020,216,Presidente,Fernando Negrão,"Retomando as declarações políticas, dou agora ...",declarações políticas 6,17 DE SETEMBRO DE 2020declarações políticas 6
217,DAR-001,17 DE SETEMBRO DE 2020,217,João Gonçalves Pereira,CDS-PP,"Sr. Presidente, Sr.as e Srs. Deputados: No fin...",declarações políticas 6,17 DE SETEMBRO DE 2020declarações políticas 6
218,DAR-001,17 DE SETEMBRO DE 2020,218,Telmo Correia,CDS-PP,Muito bem!,declarações políticas 6,17 DE SETEMBRO DE 2020declarações políticas 6
219,DAR-001,17 DE SETEMBRO DE 2020,219,João Gonçalves Pereira,CDS-PP,Não merecerão os portugueses um pouco mais do ...,declarações políticas 6,17 DE SETEMBRO DE 2020declarações políticas 6
220,DAR-001,17 DE SETEMBRO DE 2020,220,Presidente,Fernando Negrão,A Mesa regista seis inscrições para pedidos de...,declarações políticas 6,17 DE SETEMBRO DE 2020declarações políticas 6


In [44]:
def get_related_projects(citations_df, projects_df):
    #print(citations_df['Subject'].unique())
    to_out = pd.DataFrame(columns = projects_df.columns)
    for subject in citations_df['Subject'].unique():
        #print('subject', subject)
        titles = subject.split(', ')
        for title in titles:
            to_out = to_out.append(projects_df.loc[projects_df['title'] == title])
    return to_out

projects_related_to_corruption = get_related_projects(dialogs_with_corruption_citations, projects)
print(len(projects_related_to_corruption))
projects_related_to_corruption.head()
        

5


,author,description,title,url
32,"Duarte Alves (PCP) , Bruno Dias (PCP) , Antóni...",Recomenda ao Governo a reversão da alienação d...,Projeto de Resolução 471/XIV,https://www.parlamento.pt/ActividadeParlamenta...
32,"Duarte Alves (PCP) , Bruno Dias (PCP) , Antóni...",Recomenda ao Governo a reversão da alienação d...,Projeto de Resolução 471/XIV,https://www.parlamento.pt/ActividadeParlamenta...
33,"André Silva (PAN) , Bebiana Cunha (PAN) , Inês...",Recomenda ao Governo que apresente e submeta à...,Projeto de Resolução 550/XIV,https://www.parlamento.pt/ActividadeParlamenta...
33,"André Silva (PAN) , Bebiana Cunha (PAN) , Inês...",Recomenda ao Governo que apresente e submeta à...,Projeto de Resolução 550/XIV,https://www.parlamento.pt/ActividadeParlamenta...
33,"André Silva (PAN) , Bebiana Cunha (PAN) , Inês...",Recomenda ao Governo que apresente e submeta à...,Projeto de Resolução 550/XIV,https://www.parlamento.pt/ActividadeParlamenta...


In [46]:
def generate_statistics(all_dialogs, direct_citations_dialogs, related_dialogs, projects, direct_citations_projects, related_projects):
    people_statistics = {'name' : [], 
               'amount_of_corruption_citations' : [], 
               'amount_of_discussion_participations' : [],
                'percentage_of_discussion_participations' : []}
    party_statistics = {'name' : [], 
               'amount_of_corruption_citations' : [], 
               'amount_of_discussion_participations' : []}
    projects_statistics = {'total_amount_of_projects' : 0,
                'amount_of_projects_with_direct_citations' : 0,
                'amount_of_related_projects' : 0,
                'percentage_of_corruption_related_projects' : 0.,
                'people_involved' : []}
    # PEOPLE
    people_statistics['name'] = dialogs['Person'].unique()
    total_participations = len(dialogs_related_to_corruption)
    for name in people_statistics['name']:
        people_statistics['amount_of_corruption_citations'].append(len(direct_citations_dialogs.loc[direct_citations_dialogs['Person'] == name]))
        people_statistics['amount_of_discussion_participations'].append(len(dialogs_related_to_corruption.loc[dialogs_related_to_corruption['Person'] == name]))
        people_statistics['percentage_of_discussion_participations'].append(float(people_statistics['amount_of_discussion_participations'][-1])/total_participations)
    # PARTIES
    party_statistics['name'] = dialogs['Party'].unique()
    for name in party_statistics['name']:
        party_statistics['amount_of_corruption_citations'].append(len(direct_citations_dialogs.loc[direct_citations_dialogs['Party'] == name]))
        party_statistics['amount_of_discussion_participations'].append(len(dialogs_related_to_corruption.loc[dialogs_related_to_corruption['Party'] == name]))
    # PROJECTS
    projects_statistics['amount_of_related_projects'] = len(projects_related_to_corruption)
    projects_statistics['amount_of_projects_with_direct_citations'] = len(projects_with_corruption_citations)
    projects_statistics['total_amount_of_projects'] = len(projects)
    projects_statistics['percentage_of_corruption_related_projects'] = float(projects_statistics['amount_of_related_projects'])/projects_statistics['total_amount_of_projects']
    for authors_list in projects_related_to_corruption['author'].unique():
        for author in authors_list.split(','):
            if(author not in projects_statistics['people_involved']):
                projects_statistics['people_involved'].append(author)
    return pd.DataFrame.from_dict(people_statistics), pd.DataFrame.from_dict(party_statistics), projects_statistics
 
people_statistics, party_statistics, projects_statistics = generate_statistics(dialogs, dialogs_with_corruption_citations, dialogs_related_to_corruption, projects, projects_with_corruption_citations, projects_related_to_corruption)

print('PROJETOS')
print(projects_statistics)
print('PESSOAS')
people_statistics

PROJETOS
{'total_amount_of_projects': 34, 'amount_of_projects_with_direct_citations': 1, 'amount_of_related_projects': 5, 'percentage_of_corruption_related_projects': 0.14705882352941177, 'people_involved': ['Duarte Alves (PCP) ', ' Bruno Dias (PCP) ', ' António Filipe (PCP) ', ' Paula Santos (PCP) ', ' Diana Ferreira (PCP) ', ' João Dias (PCP) ', ' Alma Rivera (PCP) ', ' Jerónimo de Sousa (PCP) ', ' Vera Prata (PCP) ', ' Ana Mesquita (PCP)', 'André Silva (PAN) ', ' Bebiana Cunha (PAN) ', ' Inês de Sousa Real (PAN)']}
PESSOAS


,name,amount_of_corruption_citations,amount_of_discussion_participations,percentage_of_discussion_participations
0,Presidente,5,1030,0.356895
1,Secretária,0,10,0.003465
2,Luís Moreira Testa,0,8,0.002772
3,Ana Paula Vitorino,0,2,0.000693
4,Santinho Pacheco,0,0,0.000000
...,...,...,...,...
288,IvanGonçalves,0,0,0.000000
289,Martina Jesus,0,0,0.000000
290,Ascenso Simões,0,0,0.000000
291,AnaRitaBessa,0,0,0.000000


In [9]:
print('PARTIDOS')
party_statistics.head()


PARTIDOS


,name,amount_of_corruption_citations,amount_of_discussion_participations
0,NaN,0,0
1,Maria da Luz Rosinha,0,0
2,PS,2,48
3,PEV,2,14
4,CH,4,17


Dirijo um abraço a todos neste regresso dos Plenários à Casa da democracia. Esperávamos que nesta altura já pudéssemos ter regras mais flexíveis, mas, infelizmente, os números e as consequências concretas não nos permitem tal, e, portanto, continuamos, no essencial, com as regras que presidiram aos últimos Plenários da 1.ª Sessão Legislativa. Srs. Deputados, da nossa ordem do dia constam declarações políticas. Porém, antes disso, a Sr.ª Secretária Maria da Luz Rosinha fará o favor de anunciar a entrada de algumas iniciativas. Tem a palavra, Sr.ª Secretária.
Muito obrigado, Sr.ª Secretária Maria da Luz Rosinha. Estamos, pois, em condições de iniciar o primeiro ponto da nossa agenda, que consiste em declarações políticas. A primeira declaração política cabe ao Grupo Parlamentar do PS. Antes de dar a palavra ao Sr. Deputado Luís Moreira Testa, relembro que as regras de funcionamento do Plenário continuam a ser as mesmas que estavam em vigor antes das férias — infelizmente, mas é esta a re

Sr.as e Srs. Deputados, dou ainda conta da ordem de trabalhos da próxima reunião plenária, que se realizará no dia 6 de janeiro, quarta-feira. Do primeiro ponto consta a apresentação e discussão do Relatório sobre a Aplicação da Declaração do Estado de Emergência no período de 9 a 23 de dezembro de 2020. No segundo ponto, procederemos a um debate sobre o pedido de autorização da renovação do estado de emergência. Em terceiro lugar, apreciaremos a Petição n.º 68/XIV/1.ª (José Miguel Correia de Freitas Salgado da Cunha e outros) — Contra a eliminação da melhoria da classificação final da disciplina por exame. No quarto ponto, haverá lugar à discussão conjunta da Petição n.º 85/XIV/1.ª (António Ricardo Diegues da Silva e outros) — Máscaras para todos – Uso obrigatório de máscara facial na comunidade e dos Projetos de Resolução n.os 380/XIV/1.ª (CH) — Pela imediata distribuição de máscaras a todas as forças de segurança, segurança privada e guardas prisionais e 384/XIV/1.ª (CH) — Pela obri

In [ ]:
'''
passagem de assuntos:
    presidente que define
    Declarações políticas: 
        textos grandes ?, 
        cita 'declaração(ões) política(s)',
        cita nome do partido que vai falar,
        NÃO TEM 'esclarecimentos'
'''